In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
import gc

Using TensorFlow backend.


In [14]:
# train = pd.read_csv('../data/OHE/train.csv')
train = pd.read_csv('../data/OHE/train.csv')
y_train = pd.read_csv('../data/raw/train_labels.csv', usecols=['heart_disease_present'])
# test = pd.read_csv('../data/OHE/test.csv')
test = pd.read_csv('../data/OHE/test.csv')

In [15]:
X_train = train.loc[:,[c for c in train.columns if c != 'patient_id']]
X_test = test.loc[:,[c for c in test.columns if c != 'patient_id']]

In [16]:
X_train.head()

,slope_of_peak_exercise_st_segment_1,slope_of_peak_exercise_st_segment_2,slope_of_peak_exercise_st_segment_3,thal_fixed_defect,thal_normal,thal_reversible_defect,resting_blood_pressure_94,resting_blood_pressure_100,resting_blood_pressure_101,resting_blood_pressure_102,...,max_heart_rate_achieved_186,max_heart_rate_achieved_187,max_heart_rate_achieved_188,max_heart_rate_achieved_190,max_heart_rate_achieved_192,max_heart_rate_achieved_194,max_heart_rate_achieved_195,max_heart_rate_achieved_202,exercise_induced_angina_0,exercise_induced_angina_1
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
callbacks = [EarlyStopping(monitor='val_loss',
                           min_delta=0.0,
                           patience=1,
                           verbose=0,
                           mode='min',
                           restore_best_weights=True)]

In [7]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(X_train.shape[0], input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
#     model.add(Dense(X_train.shape[0]*2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    return model

In [7]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=30, batch_size=8, verbose=2, validation_split=0.33,
                           callbacks=callbacks)

In [8]:
model = create_baseline()

Instructions for updating:
Colocations handled automatically by placer.


In [41]:
estimator.fit(X_train, y_train)

Train on 120 samples, validate on 60 samples
Epoch 1/30
 - 0s - loss: 0.6697 - val_loss: 0.6617
Epoch 2/30
 - 0s - loss: 0.6291 - val_loss: 0.6396
Epoch 3/30
 - 0s - loss: 0.5863 - val_loss: 0.5996
Epoch 4/30
 - 0s - loss: 0.5410 - val_loss: 0.5673
Epoch 5/30
 - 0s - loss: 0.4975 - val_loss: 0.5388
Epoch 6/30
 - 0s - loss: 0.4635 - val_loss: 0.5161
Epoch 7/30
 - 0s - loss: 0.4266 - val_loss: 0.5075
Epoch 8/30
 - 0s - loss: 0.4114 - val_loss: 0.4997
Epoch 9/30
 - 0s - loss: 0.3925 - val_loss: 0.4993
Epoch 10/30
 - 0s - loss: 0.3821 - val_loss: 0.4992
Epoch 11/30
 - 0s - loss: 0.3715 - val_loss: 0.4990
Epoch 12/30
 - 0s - loss: 0.3663 - val_loss: 0.4995


In [28]:
y_preds = estimator.predict_proba(X_test)[:,1]

In [29]:
y_preds[:5]

array([0.34716144, 0.04737139, 0.85490656, 0.11767995, 0.8636585 ],
      dtype=float32)

In [48]:
sub = pd.DataFrame({'patient_id': test['patient_id'], 'heart_disease_present': y_preds})

In [49]:
sub.head()

,patient_id,heart_disease_present
0,olalu7,0.062572
1,z9n6mx,0.075576
2,5k4413,0.896371
3,mrg7q5,0.005589
4,uki4do,0.995853


In [50]:
sub.to_csv('../submissions/keras_3.csv', index=False)

# CV

In [17]:
callbacks = [EarlyStopping(monitor='val_loss',
                           min_delta=0,
                           patience=1,
                           verbose=0,
                           mode='min',
                           restore_best_weights=True)]

In [18]:
def ll(y_true, y_pred):
    return log_loss(y_true, y_pred)

In [22]:
model_name = 'keras'
def create_baseline():
    model = Sequential()
    model.add(Dense(24, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(16))
#     model.add(Dense(8))
#     model.add(Dense(4))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',)#adam
    return model

sgd = SGD(lr = 0.01, momentum = 0.9)

In [23]:
train_ids = X_train.index
k1 = 5
k2 = 5
skf1 = StratifiedKFold(n_splits=k1, shuffle=True, random_state=42)
skf2 = StratifiedKFold(n_splits=k2, shuffle=True, random_state=42)

In [24]:
counter1 = 1
y_preds = np.zeros(X_test.shape[0])
be = 0
stack = pd.DataFrame()
stack2 = pd.DataFrame()
for train_index, test_index in skf1.split(train_ids, y_train):
    counter2 = 1
    print('Fold k1 {}\n'.format(counter1))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    X_fit_ids = X_fit.index
    
    y_preds_stack = np.zeros(X_val.shape[0])
    
    estimator = KerasClassifier(build_fn=create_baseline, epochs=100,
                                batch_size=8, verbose=2, validation_data=(X_val, y_val),
                                callbacks=callbacks)
        
    model = create_baseline()
    
    for train_index2, test_index2 in skf2.split(X_fit_ids, y_fit):
        
#         print('Fold k2 {}\n'.format(counter2))
        
        X_fit2, X_val2 = X_fit.iloc[train_index2, :], X_fit.iloc[test_index2, :]
        y_fit2, y_val2 = y_fit.iloc[train_index2], y_fit.iloc[test_index2]


        estimator.fit(X_fit,
                      y_fit)

        y_preds += estimator.predict_proba(X_test)[:,1]
        y_preds_stack += estimator.predict_proba(X_val)[:,1]

        
        del X_fit2
        del X_val2
        del y_fit2
        del y_val2
        del train_index2
        del test_index2
        gc.collect()

    #     print('Guardamos el modelo')
    #     joblib.dump(lgb_model, '../saved_models/{}_{}.pkl'.format(model_name, counter))

    #     ft_importances += lgb_model.feature_importances_

        counter2 += 1
        
    stack2['patient_id'] = train['patient_id'][test_index].tolist()
    stack2['heart_disease_present'] = y_preds_stack/k2
    stack = pd.concat([stack, stack2], axis = 0)
    
    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

    counter1 += 1
    
y_preds = y_preds / (k1*k2)

print('\n\nBEST SCORE MEAN:', be / (k1*k2))
stack.to_csv('../stacking/data/train_{}.csv'.format(model_name), index=False)
sub = pd.DataFrame({'patient_id': test['patient_id'], 'heart_disease_present': y_preds})
sub.to_csv('../stacking/data/test_{}.csv'.format(model_name), index=False)

Fold k1 1

Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 1s - loss: 0.6846 - val_loss: 0.6749
Epoch 2/100
 - 0s - loss: 0.6649 - val_loss: 0.6522
Epoch 3/100
 - 0s - loss: 0.6287 - val_loss: 0.6079
Epoch 4/100
 - 0s - loss: 0.5637 - val_loss: 0.5398
Epoch 5/100
 - 0s - loss: 0.4705 - val_loss: 0.4652
Epoch 6/100
 - 0s - loss: 0.3743 - val_loss: 0.4150
Epoch 7/100
 - 0s - loss: 0.3052 - val_loss: 0.4037
Epoch 8/100
 - 0s - loss: 0.2483 - val_loss: 0.4019
Epoch 9/100
 - 0s - loss: 0.2103 - val_loss: 0.4186
Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 1s - loss: 0.6920 - val_loss: 0.6848
Epoch 2/100
 - 0s - loss: 0.6797 - val_loss: 0.6660
Epoch 3/100
 - 0s - loss: 0.6491 - val_loss: 0.6237
Epoch 4/100
 - 0s - loss: 0.5831 - val_loss: 0.5418
Epoch 5/100
 - 0s - loss: 0.4797 - val_loss: 0.4473
Epoch 6/100
 - 0s - loss: 0.3701 - val_loss: 0.3858
Epoch 7/100
 - 0s - loss: 0.2882 - val_loss: 0.3604
Epoch 8/100
 - 0s - loss: 0.2362 - val_loss: 0.3630
Train on 144 sa

Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 2s - loss: 0.6908 - val_loss: 0.6854
Epoch 2/100
 - 0s - loss: 0.6782 - val_loss: 0.6677
Epoch 3/100
 - 0s - loss: 0.6487 - val_loss: 0.6250
Epoch 4/100
 - 0s - loss: 0.5856 - val_loss: 0.5602
Epoch 5/100
 - 0s - loss: 0.4972 - val_loss: 0.4780
Epoch 6/100
 - 0s - loss: 0.3937 - val_loss: 0.3988
Epoch 7/100
 - 0s - loss: 0.3138 - val_loss: 0.3489
Epoch 8/100
 - 0s - loss: 0.2531 - val_loss: 0.3302
Epoch 9/100
 - 0s - loss: 0.2132 - val_loss: 0.3278
Epoch 10/100
 - 0s - loss: 0.1810 - val_loss: 0.3340
Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 2s - loss: 0.6894 - val_loss: 0.6847
Epoch 2/100
 - 0s - loss: 0.6763 - val_loss: 0.6675
Epoch 3/100
 - 0s - loss: 0.6471 - val_loss: 0.6319
Epoch 4/100
 - 0s - loss: 0.5938 - val_loss: 0.5652
Epoch 5/100
 - 0s - loss: 0.5047 - val_loss: 0.4801
Epoch 6/100
 - 0s - loss: 0.3972 - val_loss: 0.4014
Epoch 7/100
 - 0s - loss: 0.3136 - val_loss: 0.3530
Epoch 8/100
 - 0s - loss:

In [21]:
counter = 1
y_preds = np.zeros(X_test.shape[0])
stack = pd.DataFrame()
stack2 = pd.DataFrame()
for train_index, test_index in skf.split(train_ids, y_train):
    print('Fold {}\n'.format(counter))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    estimator = KerasClassifier(build_fn=create_baseline, epochs=100,
                                batch_size=8, verbose=2, validation_data=(X_val, y_val),
                                callbacks=callbacks)
    model = create_baseline()
    

    estimator.fit(X_fit,
                  y_fit)
    
    y_preds += estimator.predict_proba(X_test)[:,1]
    stack2['patient_id'] = train['patient_id'][test_index].tolist()
    stack2['heart_disease_present'] = estimator.predict_proba(X_val)[:,1]
    stack = pd.concat([stack, stack2], axis = 0)
#     print(log_loss(y_val, estimator.predict_proba(X_val)))

    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

#     print('Guardamos el modelo')
#     joblib.dump(lgb_model, 'saved_models/{}_{}.pkl'.format(model_name, counter))

    counter += 1
    
y_preds = y_preds / (counter-1)

Fold 1

Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 1s - loss: 0.6915 - val_loss: 0.6876
Epoch 2/100
 - 0s - loss: 0.6796 - val_loss: 0.6714
Epoch 3/100
 - 0s - loss: 0.6498 - val_loss: 0.6282
Epoch 4/100
 - 0s - loss: 0.5786 - val_loss: 0.5514
Epoch 5/100
 - 0s - loss: 0.4745 - val_loss: 0.4565
Epoch 6/100
 - 0s - loss: 0.3579 - val_loss: 0.3929
Epoch 7/100
 - 0s - loss: 0.2782 - val_loss: 0.3759
Epoch 8/100
 - 0s - loss: 0.2272 - val_loss: 0.3699
Epoch 9/100
 - 0s - loss: 0.1904 - val_loss: 0.3812
Fold 2

Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 1s - loss: 0.6876 - val_loss: 0.6858
Epoch 2/100
 - 0s - loss: 0.6697 - val_loss: 0.6730
Epoch 3/100
 - 0s - loss: 0.6299 - val_loss: 0.6473
Epoch 4/100
 - 0s - loss: 0.5538 - val_loss: 0.6152
Epoch 5/100
 - 0s - loss: 0.4495 - val_loss: 0.5986
Epoch 6/100
 - 0s - loss: 0.3384 - val_loss: 0.6027
Fold 3

Train on 144 samples, validate on 36 samples
Epoch 1/100
 - 1s - loss: 0.6890 - val_loss: 0.6845
Epoch 2/1

In [14]:
stack.head()

,patient_id,heart_disease_present
0,0z64un,0.100189
1,yt1s1x,0.889463
2,3nwy2n,0.910203
3,1r508r,0.154261
4,cvux3j,0.134719


In [15]:
stack.to_csv('../stacking/data/train_{}.csv'.format(model_name), index=False)

In [16]:
y_preds[:5]

array([0.32616482, 0.06583074, 0.81569513, 0.14843071, 0.84419267])

In [17]:
sub = pd.DataFrame({'patient_id': test['patient_id'], 'heart_disease_present': y_preds})

In [18]:
sub.to_csv('../stacking/data/test_{}.csv'.format(model_name), index=False)